In [2]:
# !unzip ./train_metadata.json.zip

Archive:  ./train_metadata.json.zip
  inflating: train_metadata.json     
  inflating: __MACOSX/._train_metadata.json  


In [3]:
# from google.colab import drive

# drive.mount('/content/drive')

In [18]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import json

from sklearn.model_selection import train_test_split

In [19]:
#with open('../input/herbarium-2022-fgvc9/train_metadata.json', 'rb') as f:
with open('./train_metadata.json', 'rb') as f:
    train_meta = json.load(f)

In [20]:
df = pd.DataFrame(train_meta['categories'])

In [21]:
maps = list(enumerate(df['family'].unique()))
maps = {k: v for (v,k) in maps}
df['family_id'] = df['family'].map(maps)

In [22]:
cat_fam = df[['category_id', 'family_id']]
cat_fam['family_id'] = cat_fam['family_id'].astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [23]:
df = pd.DataFrame(train_meta['annotations'])
df = df[['category_id', 'image_id']]

images = pd.DataFrame(train_meta['images'])

dataset1 = df.merge(cat_fam, on='category_id', how='left').merge(images, on='image_id', how='left')
dataset1 = dataset1[['file_name', 'category_id', 'family_id']]

In [24]:
dataset_path = '/content/dataset_fsplit/'
os.mkdir(dataset_path)

grouped = dataset1.groupby('family_id')

for name, group in grouped:
    subgroups = group.groupby('category_id')
    if subgroups.ngroups > 1:
        os.mkdir(dataset_path + str(name))
        train_X, validation_X, train_Y, validation_Y = train_test_split(group['file_name'], group['category_id'], test_size=0.05, stratify=group['category_id'], random_state=0)

        train_X.to_csv(f'{dataset_path}{name}/train_X.csv')
        validation_X.to_csv(f'{dataset_path}{name}/valid_X.csv')
        train_Y.to_csv(f'{dataset_path}{name}/train_Y.csv')
        validation_Y.to_csv(f'{dataset_path}{name}/valid_Y.csv')
    else:
        continue

In [ ]:
!zip -r -9 dataset_fsplit.zip ./dataset_fsplit

Export to csv

In [26]:
!cp dataset_fsplit.zip /content/drive/MyDrive/kaggle_Herbarium22/